In [ ]:
workdir_ = '/glade/work/juliob/'
import sys
sys.path.append(workdir_ + 'myPythonTools/GenlTools/')
sys.path.append(workdir_ + 'myPythonTools/Utils/')
sys.path.append(workdir_ + 'PyRegridding/Regridder/')
sys.path.append(workdir_ + 'PyRegridding/Utils/')

# Own local packages
import AveragingUtils as Av
import VertRegridFlexLL as Vrg
import PlotUtil as Pu
import validation_data as Val

# The usual
from datetime import date
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.tri as tri
from matplotlib.colors import ListedColormap, BoundaryNorm

# Cartopy for pretty maps
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# Some other useful packages 
import importlib
import copy
import time
import cftime

importlib.reload( Pu )
importlib.reload(Av)
importlib.reload(Val)


In [ ]:
#####################
user_C,user_X='juliob','juliob'

subd='regridded'
htap='cam.h0'

exp_C = 'fmtndg2000_MM_control'  # 'fmtndg_S2_x04' #_rd1p0_cd0p1'
exp_X = 'fmtndg2000_MM_NOblj'  # 'fmtndg_S2_x04' #_rd1p0_cd0p1'
exp_X = 'fmtndg2000_MM_x21_2'  # 'fmtndg_S2_x04' #_rd1p0_cd0p1'

path_C = '/glade/derecho/scratch/juliob/archive/'+exp_C+'/atm/'+ subd +'/'+exp_C+'.'+ htap + '.*.nc' 
path_X = '/glade/derecho/scratch/juliob/archive/'+exp_X+'/atm/'+ subd +'/'+exp_X+'.'+ htap + '.*.nc' 

###
trim_years_c, nyr_max_c = True, 1000
trim_years_x, nyr_max_x = True, 1000


Dc = xr.open_mfdataset( path_C ,data_vars='different', coords='different' )
Dx = xr.open_mfdataset( path_X ,data_vars='different', coords='different' )


if ( trim_years_c==True ):
    nyr=np.minimum( nyr_max_c*12 , 12*(len(Dc.time)//12) )
    print(f"Discarding last {len(Dc.time)-nyr} months of {exp_C}" )
    Dc=Dc.isel(time=np.arange(nyr))

if ( trim_years_x==True ):
    nyr=np.minimum( nyr_max_x*12 , 12*(len(Dx.time)//12) )
    print(f"Discarding last {len(Dx.time)-nyr} months of {exp_X}" )
    Dx=Dx.isel(time=np.arange(nyr))


zlev=-7.0*np.log( Dx.lev.values/1_000. )


sqP = np.sqrt(Dx.UPWP_CLUBB**2+Dx.VPWP_CLUBB**2 )
Dx['MOMFLXMAG_CLUBB'] = sqP

In [ ]:
season='SON'
fld='Nudge_U'
#fld='UTEND_GWDTOT'
scale=86400.

AAs_C,years_c,months_c = Av.Seasonal( ds=Dc, season=season , fld=fld ,return_time=True)
AAs_X,years_x,months_x = Av.Seasonal( ds=Dx, season=season , fld=fld ,return_time=True)
UUs_X,years_x,months_x = Av.Seasonal( ds=Dx, season=season , fld='U' ,return_time=True)

AAs_C = scale*AAs_C
AAs_X = scale*AAs_X

AAsx_C=np.average( AAs_C , axis=2 )
AAsx_X=np.average( AAs_X , axis=2 )
UUsx_X=np.average( UUs_X , axis=2 )


#kl0,kl1=9,10 #11,12
kl0,kl1=11,12
kl0,kl1=7,9
#kl0,kl1=52,60


In [ ]:


fig = plt.figure(figsize=(20, 7))

#AAyz_x = 86400.*np.average(Dx.Nudge_U[mon,:,:,:],axis=2)
#AAyz_c = 86400.*np.average(Dc.Nudge_U[mon,:,:,:],axis=2)
ndglev=5*np.linspace( -2,2,num=21) 

ndglev = np.array([-100,-50,-20,-10,-5,-4,-3,-2,-1,0,1,2,3,4,5,10,20,50,100]) /5.
ulev=np.linspace(-60,120,num=19)

# Define a custom colormap with linear color transitions
colors = plt.get_cmap('bwr')(np.linspace(0, 1, len(ndglev) - 1))
cmap = ListedColormap(colors)

# Create a BoundaryNorm to map values to colors
norm = BoundaryNorm(ndglev, len(ndglev) - 1)

Axes = Pu.axes_def(n=1,nxplo=3,nyplo=1 ) 
ax1 = fig.add_axes( Axes )

co2=ax1.contourf(Dx.lat, zlev, AAsx_C ,levels=ndglev,norm=norm,cmap=cmap)
co3=ax1.contour(Dx.lat, zlev, UUsx_X ,levels=ulev,colors='black')
ax1.set_ylim( (0,65))
ax1.set_title( f"{season.upper()}  {years_c[0]}-{years_c[-1]} {fld}  {exp_C}" , fontsize=12)

cbar = plt.colorbar(co2, shrink=.6)

Axes = Pu.axes_def(n=2,nxplo=3,nyplo=1 ) 
ax1 = fig.add_axes( Axes )

co2=ax1.contourf(Dx.lat, zlev, AAsx_X ,levels=ndglev,norm=norm,cmap=cmap)
co3=ax1.contour(Dx.lat, zlev, UUsx_X ,levels=ulev,colors='black')
ax1.set_ylim( (0,65))
ax1.set_title( f"{season.upper()}  {years_c[0]}-{years_c[-1]} {fld} {exp_X}" , fontsize=12)

cbar = plt.colorbar(co2, shrink=.6)

Axes = Pu.axes_def(n=3,nxplo=3,nyplo=1 ) 
ax1 = fig.add_axes( Axes )

co2=ax1.contourf(Dx.lat, zlev, AAsx_X-AAsx_C ,levels=ndglev,norm=norm,cmap=cmap)
co3=ax1.contour(Dx.lat, zlev, UUsx_X ,levels=ulev,colors='black')
ax1.set_ylim( (0,65))
ax1.set_title(f' {exp_X} - {exp_C}')

cbar = plt.colorbar(co2, shrink=.6)

#fig.suptitle( f'{exp_X}')



In [ ]:


Zl1,Zl0=49.,55.

kl0=np.argmin( np.abs(  zlev - Zl0 ) )
kl1=np.argmin( np.abs(  zlev - Zl1 ) )

AAsz_C = np.average( AAs_C[kl0:kl1,:,:],axis=0)
AAsz_X = np.average( AAs_X[kl0:kl1,:,:],axis=0)




######################
# Pretty latlon plot
######################

#MapProj = ccrs.Robinson(central_longitude=180.)
MapProj = ccrs.PlateCarree(central_longitude=180.)
DataProj = ccrs.PlateCarree()

fig = plt.figure(figsize=(20, 12))
   
ndglev=10*np.linspace( -2,2,num=21) /1.
npo=0

zA = f" ( {Dx.lev[kl0]:.0f}-{Dx.lev[kl1]:.0f} hPa, {zlev[kl1]:.0f}-{zlev[kl0]:.0f} km)"
CoastColors=['black','black','black','black','black','black']

npo=npo+1
Axes = Pu.axes_def(n=npo,nxplo=2,nyplo=2 ) 
ax1 = fig.add_axes( Axes , projection=MapProj)
ax1.set_global()
ax1.coastlines(resolution='110m',color=CoastColors[npo-1])
co2=ax1.contourf(Dx.lon,Dx.lat, AAsz_C ,transform=DataProj,levels=ndglev,cmap='bwr', extend='both')
cbar = plt.colorbar(co2, shrink=.6)
ax1.set_title( f"{season.upper()}  {years_c[0]}-{years_c[-1]} {fld} {zA}  {exp_C}" , fontsize=16)

npo=npo+1
Axes = Pu.axes_def(n=npo,nxplo=2,nyplo=2 ) 
ax1 = fig.add_axes( Axes , projection=MapProj)
ax1.set_global()
ax1.coastlines(resolution='110m',color=CoastColors[npo-1])
co2=ax1.contourf(Dx.lon,Dx.lat, AAsz_X ,transform=DataProj,levels=ndglev,cmap='bwr', extend='both')
cbar = plt.colorbar(co2, shrink=.6)
ax1.set_title( f"{season.upper()}  {years_x[0]}-{years_x[-1]} {fld} {zA}  {exp_X}" , fontsize=16)

npo=npo+1

if (fld == 'Nudge_U'):
    Axes = Pu.axes_def(n=npo,nxplo=2,nyplo=2 ) 
    print(f" {npo} {Axes} ")
    ax1 = fig.add_axes( Axes , projection=MapProj)
    ax1.set_global()
    ax1.coastlines(resolution='110m',color=CoastColors[npo-1])
    co2=ax1.contourf(Dx.lon,Dx.lat, np.abs(AAsz_C) - np.abs(AAsz_X) ,transform=DataProj,levels=ndglev ,cmap='bwr_r', extend='both')
    cbar = plt.colorbar(co2, ax=ax1 , shrink=0.6 )# , extend='both' ,extendfrac=0.1)
    #ax1.set_title( f"{season.upper()}  {years_c[0]}-{years_c[-1]} Nudge_U {zA} Diff " , fontsize=16)
    ax1.set_title(f' ||Test|| - ||Control||')
    
    ax1.text(1.15, 0.1, f"{exp_C} was better", fontsize=12, ha='left', va='center', 
                bbox=dict(facecolor='white', edgecolor='black') , 
                transform=ax1.transAxes, rotation=0.)  #xycoords='axes fraction')
    ax1.text(1.15, 0.9, f"{exp_X} is better", fontsize=12, ha='left', va='center', 
                bbox=dict(facecolor='white', edgecolor='black') , 
                transform=ax1.transAxes, rotation=0.)  #xycoords='axes fraction')
    
    ax1.annotate('', xy=(1.2, 0.8), xytext=(1.2, 0.6), 
                 arrowprops=dict(arrowstyle='->', lw=2),
                 xycoords='axes fraction', textcoords='axes fraction',fontsize=12)
    
    ax1.annotate('', xy=(1.2, 0.2), xytext=(1.2, 0.4), 
                 arrowprops=dict(arrowstyle='->', lw=2),
                 xycoords='axes fraction', textcoords='axes fraction',fontsize=12)
    
else:
    Axes = Pu.axes_def(n=npo,nxplo=2,nyplo=2 ) 
    print(f" {npo} {Axes} ")
    ax1 = fig.add_axes( Axes , projection=MapProj)
    ax1.set_global()
    ax1.coastlines(resolution='110m',color=CoastColors[npo-1])
    co2=ax1.contourf(Dx.lon,Dx.lat, AAsz_X - AAsz_C ,transform=DataProj,levels=ndglev/2,cmap='bwr', extend='both')
    cbar = plt.colorbar(co2, ax=ax1 , shrink=0.6 )# , extend='both' ,extendfrac=0.1)
    #ax1.set_title( f"{season.upper()}  {years_c[0]}-{years_c[-1]} Nudge_U {zA} Diff " , fontsize=16)
    ax1.set_title(f' Test - Control')


fig.suptitle(  f"Control = {exp_C}, Test = {exp_X} ", ha="left", fontsize=16, x=0.0 )


In [ ]:
#################











##############